# HAP-E group
# Meeting #05 Intro to statistics in Python

## HEADER
Who: Ed Harris <br>
What: A basic data analysis example using Python <br>
When: 2021-08-06 <br>
<br>

## CONTENTS
00 Setup <br>
01 correlation <br>
02 regression <br>
03 1-way ANOVA <br>
<br>

## 00 Setup

### Libraries

You will almost always load up a few libraries and have some setup code for every python data analysis script.  Setting up your own template script or cheatsheet might be a good idea.

`Pandas` for data analysis <br>
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html

`NumPy` for numeric manipulation <br>
https://numpy.org/doc/stable/user/quickstart.html

`Matplotlib` for graphing and visualization <br>
https://matplotlib.org/stable/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py

`SciPy` for most mainstream stats functions <br>
https://docs.scipy.org/doc/scipy/reference/tutorial/general.html

In [ ]:
# import and "alias" the usual suspects**
# **assumes these are already installed
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt # NB subset of matplotlib pyplot
import scipy.stats
import seaborn as sns # fancy plots

# for 1-way ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

# for displaying plots "in-line" in notebooks
%matplotlib inline 

### Read in some data

The dataset we will use is in Tidy Data format [(see Wickham 2014)](https://vita.had.co.nz/papers/tidy-data.pdf).

The data are in an Excel file `potatoes.xlsx` with the first tab containing the data and the second tab containing a data dictionary, describing the variables in a reproducible way.

`variety` the potato variety - 3 types of potato are in this dataset

`plant_count` the number of plants per sample

`stem_count` the number of stems per sample

`tuber_count` the (mean) number of potatoes per m^2

This is part of a real, bigger dataset looking at growth patterns in potato yield.


Some generic code for reading in data

For a local file:

`df = pd.read_excel (r'Path where the Excel file is stored\File name.xlsx', sheet_name='your Excel sheet name')` <br>
`print (df)`

For a web file:

`df = pd.read_excel ('http://URL for your data.com/file_name.xlsx')` <br>
`print (df)`

<br>

In [ ]:
# Read in the data

tattiedat = pd.read_excel("potatoes.xlsx")

# Examine the data object
# NB '.head()' is an "attribute" function of the data object 
tattiedat.head()

In [ ]:
# Coarse summary of numeric variables
tattiedat.describe()

In [ ]:
# Coarse summary of character variables
tattiedat.describe(include = ['object'])

<br>
Maybe you would like a frequency summary of a categorical variable?

In [ ]:
tattiedat.value_counts('variety')

### Histogram

Graph the distribution of a numeric variable

In [ ]:
# remember plt is matplotlib.pyplot
# using the hist() function on mumeric data
# print(tattiedat['stem_count'])
plt.hist(tattiedat['stem_count'])

<br>
We can make the plot prettier if required for more than just exploring the data.

In [ ]:
tattiedat.describe()['stem_count']

In [ ]:
# Beautifying functions for plots
plt.hist(tattiedat['stem_count'])
plt.xlabel('Stem count per sample')
plt.ylabel('Frequency')
plt.title('Distribution of stem count for \n several potato varieties ')
plt.ylim(ymax = 40)
plt.text(x = 35, y = 30, s = 'mean = 25.9')

### Barplot

Barplot for the count data potato varieties. 

In [ ]:
tattiedat.value_counts('variety')

In [ ]:
var = ['Maris Piper', 'Georgina', 'Lanorma']
count = [64, 50, 27]
plt.bar(var, count)
plt.xlabel('Potato variety')
plt.ylabel('Frequency')
plt.title('N observations for each potato variety ')

## 01 correlation

Let's look at a simple correlation between the `stem_count` and the `plant_count`

In [ ]:
plt.scatter(x = tattiedat['plant_count'], y = tattiedat['stem_count'])
plt.xlabel('Plant count')
plt.ylabel('Stem count')
plt.title('Relationship between stems and plants')

<br>
Looking at the plot, we would expect a strong, significant positive correlation.

In [ ]:
# calculate the correlation and test it
# NB the data type conversion np.array()
mycor = scipy.stats.pearsonr(x = np.array(tattiedat['plant_count']), 
                     y = np.array(tattiedat['stem_count']))
mycor

In [ ]:
print('The correlation coefficient is r = ', round(mycor[0], 3), '\n', 
      'The P-value is P = ', round(mycor[1], 15))

## 02 regression

Let's perform regression to predict the tuber number from the stem number.

In [ ]:
# do the regression
lm0 = scipy.stats.linregress(x = tattiedat['stem_count'], 
                       y = tattiedat['tuber_count'])

# ugly
lm0 

In [ ]:
# pretty
print('The intercept (std err) = ', 
      round(lm0.intercept,3), '(', 
      round(lm0.intercept_stderr,3), ')')

print('The slope (std err) = ', 
      round(lm0.slope,3), '(', 
      round(lm0.stderr,3), ')')

print('The R-squared value =  ', 
      round(lm0.rvalue,3))

print('The P-value =  ', 
      round(lm0.pvalue,3))


In [ ]:
# base scatterplot
plt.scatter(x = tattiedat['stem_count'], y = tattiedat['tuber_count'])
plt.xlabel('Stem count')
plt.ylabel('Tuber count')
plt.title('Relationship between tubers and stems')

In [ ]:
# regression plot
sns.regplot(x = tattiedat['stem_count'],
            y = tattiedat['tuber_count'])
plt.xlabel('Stem count')
plt.ylabel('Tuber count')
plt.title('Relationship between tubers and stems')

In [ ]:
tattiedat['stem_count']

## 03 1-way ANOVA

Simple 1-way ANOVA for the number of stems as a function of potato variety.
<br>

In [ ]:
# first make a boxplot
tattiedat.boxplot('stem_count', by = 'variety')
plt.xlabel('Potato variety')
plt.ylabel('Stem count')

In [ ]:
# calculate ANOVA stem_count ~ variety
aov0 = ols('stem_count ~ variety', data = tattiedat).fit()
aov0_table = sm.stats.anova_lm(aov0, typ = 2)
print(aov0_table)